# Exercise 8: Agent Communication & Handoffs
## Building a System Where Agents Can Recommend Consulting Other Agents

In this notebook, we'll create a financial planning workflow system where agents can communicate with each other and recommend handoffs to other specialists. We'll learn how to design inter-agent communication patterns and workflow management.

## Learning Objectives
- Implement inter-agent communication patterns
- Create agent handoff mechanisms
- Design financial planning workflows
- Build systems where agents can recommend other agents
- Understand workflow orchestration
- Create collaborative agent interactions

Let's build a collaborative financial planning system! 🤖🤝📊

## Setup and Imports

First, let's import all the necessary libraries for building our inter-agent communication system.

In [1]:
# Install required packages
!uv add langchain langchain-openai langchain-community python-dotenv yfinance pandas numpy

Resolved 393 packages in 0.95ms
Audited 224 packages in 0.09ms


In [2]:
# Import necessary libraries
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool, StructuredTool, tool
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv
import json
import math
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

print("✓ Setup complete!")
print("✓ Ready to build agent communication system!")

✓ Setup complete!
✓ Ready to build agent communication system!


## Part 1: Enhanced Tax Advisor Tools with Communication

Let's start by creating enhanced tools for our Tax Advisor agent that can recommend handoffs to other specialists and communicate findings that require other expertise.

In [3]:
# Enhanced Tax calculation tools with communication capabilities

# Communication tracking system
class AgentCommunication:
    def __init__(self):
        self.messages = []
        self.handoff_requests = []
    
    def add_message(self, from_agent: str, message: str, recommendations: List[str] = None):
        self.messages.append({
            'from_agent': from_agent,
            'message': message,
            'recommendations': recommendations or [],
            'timestamp': datetime.now().isoformat()
        })
    
    def request_handoff(self, from_agent: str, to_agent: str, reason: str, data: dict):
        self.handoff_requests.append({
            'from_agent': from_agent,
            'to_agent': to_agent,
            'reason': reason,
            'data': data,
            'timestamp': datetime.now().isoformat()
        })

# Global communication system
comm_system = AgentCommunication()

@tool
def enhanced_tax_analysis(input_data: str) -> str:
    """
    Enhanced tax analysis that can recommend other specialists.
    
    Args:
        input_data: JSON string with income, filing_status, deductions, and other financial info
    
    Returns:
        Tax analysis with recommendations for other specialists
    """
    try:
        data = json.loads(input_data)
        income = data['income']
        filing_status = data['filing_status']
        deductions = data.get('deductions', 0)
        has_investments = data.get('has_investments', False)
        monthly_income = data.get('monthly_income', income / 12)
        
        # Basic tax calculation (simplified)
        standard_deductions = {
            'single': 14600,
            'married_joint': 29200,
            'married_separate': 14600,
            'head_of_household': 21900
        }
        
        total_deductions = max(deductions, standard_deductions.get(filing_status, 14600))
        taxable_income = max(0, income - total_deductions)
        
        # Simple tax calculation
        tax_rate = 0.22 if income > 95000 else 0.12 if income > 44725 else 0.10
        estimated_tax = taxable_income * tax_rate
        
        analysis = f"""
TAX ANALYSIS RESULTS:
- Gross Income: ${income:,.2f}
- Filing Status: {filing_status}
- Estimated Tax: ${estimated_tax:,.2f}
- Effective Rate: {(estimated_tax/income*100):.1f}%

TAX OPTIMIZATION INSIGHTS:
"""
        
        # Determine recommendations for other specialists
        recommendations = []
        
        # Investment-related recommendations
        if has_investments:
            recommendations.append("Investment Analyst")
            analysis += "- Consider tax-loss harvesting strategies\n"
            analysis += "- Review tax-efficient investment accounts\n"
            
            # Add communication message
            comm_system.add_message(
                from_agent="Tax Advisor",
                message=f"Client has investments with ${income:,.2f} income. Tax-efficient strategies needed.",
                recommendations=["Investment Analyst"]
            )
        
        # Budget planning recommendations
        if monthly_income > 0:
            savings_potential = monthly_income * 0.2  # 20% savings rate
            if savings_potential > 1000:
                recommendations.append("Budget Planner")
                analysis += f"- Potential monthly savings: ${savings_potential:,.2f}\n"
                analysis += "- Consider retirement account contributions\n"
                
                # Add communication message
                comm_system.add_message(
                    from_agent="Tax Advisor",
                    message=f"Client has good savings potential (${savings_potential:,.2f}/month). Retirement planning needed.",
                    recommendations=["Budget Planner"]
                )
        
        # High earner recommendations
        if income > 100000:
            recommendations.extend(["Investment Analyst", "Budget Planner"])
            analysis += "- High earner: Consider advanced strategies\n"
            analysis += "- Estate planning may be beneficial\n"
        
        if recommendations:
            analysis += f"\n🤝 RECOMMENDED CONSULTATIONS:\n"
            for rec in set(recommendations):  # Remove duplicates
                analysis += f"- {rec}: For specialized advice\n"
        
        return analysis
    
    except Exception as e:
        return f"Error in tax analysis: {str(e)}"

@tool
def tax_investment_coordination(input_data: str) -> str:
    """
    Coordinate tax implications with investment decisions.
    
    Args:
        input_data: JSON string with investment and tax information
    
    Returns:
        Coordinated tax and investment advice
    """
    try:
        data = json.loads(input_data)
        investment_amount = data.get('investment_amount', 0)
        income = data.get('income', 0)
        
        advice = f"""
TAX-INVESTMENT COORDINATION:
- Investment Amount: ${investment_amount:,.2f}
- Current Income: ${income:,.2f}

RECOMMENDATIONS:
"""
        
        # Tax-advantaged accounts
        if income > 50000:
            advice += "- Maximize 401(k) contributions for tax reduction\n"
            advice += "- Consider Roth IRA for tax-free growth\n"
        
        # Request handoff to Investment Analyst
        comm_system.request_handoff(
            from_agent="Tax Advisor",
            to_agent="Investment Analyst",
            reason="Tax-efficient investment strategy needed",
            data={
                'investment_amount': investment_amount,
                'income': income,
                'tax_considerations': 'high_earner' if income > 100000 else 'moderate'
            }
        )
        
        advice += "\n🔄 HANDOFF REQUESTED: Investment Analyst for tax-efficient portfolio design\n"
        
        return advice
    
    except Exception as e:
        return f"Error in tax-investment coordination: {str(e)}"

# Test the enhanced tax tools
print("Testing Enhanced Tax Analysis:")
test_input = json.dumps({
    "income": 85000,
    "filing_status": "single",
    "deductions": 8000,
    "has_investments": True,
    "monthly_income": 7100
})
result = enhanced_tax_analysis.run(test_input)
print(result)
print("\n✓ Enhanced Tax tools created!")
print(f"✓ Messages in communication system: {len(comm_system.messages)}")
print(f"✓ Handoff requests: {len(comm_system.handoff_requests)}")

Testing Enhanced Tax Analysis:

TAX ANALYSIS RESULTS:
- Gross Income: $85,000.00
- Filing Status: single
- Estimated Tax: $8,448.00
- Effective Rate: 9.9%

TAX OPTIMIZATION INSIGHTS:
- Consider tax-loss harvesting strategies
- Review tax-efficient investment accounts
- Potential monthly savings: $1,420.00
- Consider retirement account contributions

🤝 RECOMMENDED CONSULTATIONS:
- Investment Analyst: For specialized advice
- Budget Planner: For specialized advice


✓ Enhanced Tax tools created!
✓ Messages in communication system: 2
✓ Handoff requests: 0


## Part 2: Enhanced Investment Analyst Tools with Communication

Now let's create enhanced Investment Analyst tools that can communicate with other agents and handle handoffs from the Tax Advisor.

In [4]:
# Enhanced Investment analysis tools with communication capabilities

@tool
def enhanced_portfolio_analysis(input_data: str) -> str:
    """
    Enhanced portfolio analysis that considers tax implications and budget constraints.
    
    Args:
        input_data: JSON string with portfolio, income, and tax information
    
    Returns:
        Portfolio analysis with cross-agent recommendations
    """
    try:
        data = json.loads(input_data)
        portfolio = data.get('holdings', {})
        total_value = data.get('total_value', 100000)
        income = data.get('income', 0)
        age = data.get('age', 35)
        
        # Calculate current allocation
        total_percentage = sum(portfolio.values())
        if total_percentage != 100:
            return "Error: Portfolio percentages must sum to 100%"
        
        stock_percentage = portfolio.get('stocks', 0)
        bond_percentage = portfolio.get('bonds', 0)
        
        analysis = f"""
ENHANCED PORTFOLIO ANALYSIS:
- Total Value: ${total_value:,.2f}
- Age: {age} years
- Current Stock Allocation: {stock_percentage}%
- Current Bond Allocation: {bond_percentage}%

INVESTMENT RECOMMENDATIONS:
"""
        
        # Age-based recommendations
        recommended_stock = max(20, 100 - age)
        
        if stock_percentage < recommended_stock - 10:
            analysis += f"- Consider increasing stock allocation to ~{recommended_stock}%\n"
        elif stock_percentage > recommended_stock + 10:
            analysis += f"- Consider reducing stock allocation to ~{recommended_stock}%\n"
        
        recommendations = []
        
        # Tax considerations
        if income > 75000:
            analysis += "- Consider tax-advantaged accounts\n"
            analysis += "- Tax-loss harvesting opportunities\n"
            recommendations.append("Tax Advisor")
            
            # Add communication message
            comm_system.add_message(
                from_agent="Investment Analyst",
                message=f"High-income client (${income:,.2f}) needs tax-efficient investment strategy",
                recommendations=["Tax Advisor"]
            )
        
        # Budget planning considerations
        monthly_investment_capacity = (income * 0.15) / 12  # 15% of income
        if monthly_investment_capacity > 500:
            analysis += f"- Monthly investment capacity: ${monthly_investment_capacity:,.2f}\n"
            analysis += "- Consider systematic investment plans\n"
            recommendations.append("Budget Planner")
            
            # Add communication message
            comm_system.add_message(
                from_agent="Investment Analyst",
                message=f"Client has ${monthly_investment_capacity:,.2f} monthly investment capacity. Need budget optimization.",
                recommendations=["Budget Planner"]
            )
        
        # Risk assessment
        if age < 35 and stock_percentage < 70:
            analysis += "- Young investor: Consider higher growth allocation\n"
        elif age > 55 and stock_percentage > 60:
            analysis += "- Pre-retirement: Consider more conservative allocation\n"
        
        if recommendations:
            analysis += f"\n🤝 RECOMMENDED CONSULTATIONS:\n"
            for rec in set(recommendations):
                analysis += f"- {rec}: For specialized coordination\n"
        
        return analysis
    
    except Exception as e:
        return f"Error in portfolio analysis: {str(e)}"

@tool
def investment_budget_coordination(input_data: str) -> str:
    """
    Coordinate investment strategy with budget planning.
    
    Args:
        input_data: JSON string with investment goals and budget information
    
    Returns:
        Coordinated investment and budget advice
    """
    try:
        data = json.loads(input_data)
        investment_goal = data.get('investment_goal', 0)
        monthly_income = data.get('monthly_income', 0)
        time_horizon = data.get('time_horizon', 10)  # years
        
        monthly_needed = investment_goal / (time_horizon * 12)
        
        advice = f"""
INVESTMENT-BUDGET COORDINATION:
- Investment Goal: ${investment_goal:,.2f}
- Time Horizon: {time_horizon} years
- Monthly Investment Needed: ${monthly_needed:,.2f}
- Current Monthly Income: ${monthly_income:,.2f}

STRATEGY:
"""
        
        affordability = (monthly_needed / monthly_income * 100) if monthly_income > 0 else 0
        
        if affordability > 20:
            advice += "- Goal may require budget optimization\n"
            advice += "- Consider extending timeline or increasing income\n"
            
            # Request handoff to Budget Planner
            comm_system.request_handoff(
                from_agent="Investment Analyst",
                to_agent="Budget Planner",
                reason="Budget optimization needed for investment goals",
                data={
                    'investment_goal': investment_goal,
                    'monthly_needed': monthly_needed,
                    'monthly_income': monthly_income,
                    'affordability_ratio': affordability
                }
            )
            
            advice += "\n🔄 HANDOFF REQUESTED: Budget Planner for expense optimization\n"
        else:
            advice += f"- Goal is achievable ({affordability:.1f}% of income)\n"
            advice += "- Consider automating investments\n"
        
        return advice
    
    except Exception as e:
        return f"Error in investment-budget coordination: {str(e)}"

# Test the enhanced investment tools
print("Testing Enhanced Portfolio Analysis:")
test_input = json.dumps({
    "holdings": {
        "stocks": 60,
        "bonds": 30,
        "cash": 10
    },
    "total_value": 200000,
    "income": 90000,
    "age": 32
})
result = enhanced_portfolio_analysis.run(test_input)
print(result)
print("\n✓ Enhanced Investment tools created!")
print(f"✓ Total messages: {len(comm_system.messages)}")
print(f"✓ Total handoff requests: {len(comm_system.handoff_requests)}")

Testing Enhanced Portfolio Analysis:

ENHANCED PORTFOLIO ANALYSIS:
- Total Value: $200,000.00
- Age: 32 years
- Current Stock Allocation: 60%
- Current Bond Allocation: 30%

INVESTMENT RECOMMENDATIONS:
- Consider tax-advantaged accounts
- Tax-loss harvesting opportunities
- Monthly investment capacity: $1,125.00
- Consider systematic investment plans
- Young investor: Consider higher growth allocation

🤝 RECOMMENDED CONSULTATIONS:
- Tax Advisor: For specialized coordination
- Budget Planner: For specialized coordination


✓ Enhanced Investment tools created!
✓ Total messages: 4
✓ Total handoff requests: 0


## Part 3: Enhanced Budget Planner Tools with Communication

Finally, let's create enhanced Budget Planner tools that can handle handoffs from other agents and coordinate comprehensive financial planning.

In [5]:
# Enhanced Budget planning tools with communication capabilities

@tool
def enhanced_budget_analysis(input_data: str) -> str:
    """
    Enhanced budget analysis that considers tax and investment implications.
    
    Args:
        input_data: JSON string with budget, income, and financial goals
    
    Returns:
        Budget analysis with cross-agent recommendations
    """
    try:
        data = json.loads(input_data)
        monthly_income = data['monthly_income']
        expenses = data.get('expenses', {})
        financial_goals = data.get('financial_goals', [])
        
        total_expenses = sum(expenses.values())
        remaining = monthly_income - total_expenses
        savings_rate = (remaining / monthly_income * 100) if monthly_income > 0 else 0
        
        analysis = f"""
ENHANCED BUDGET ANALYSIS:
- Monthly Income: ${monthly_income:,.2f}
- Total Expenses: ${total_expenses:,.2f}
- Available for Savings: ${remaining:,.2f}
- Current Savings Rate: {savings_rate:.1f}%

BUDGET OPTIMIZATION:
"""
        
        recommendations = []
        
        # Savings rate analysis
        if savings_rate < 10:
            analysis += "- CRITICAL: Savings rate too low for financial security\n"
            analysis += "- Review and reduce discretionary spending\n"
        elif savings_rate < 20:
            analysis += "- MODERATE: Increase savings rate to 20%+ if possible\n"
        else:
            analysis += "- EXCELLENT: Strong savings rate for financial growth\n"
        
        # Tax optimization opportunities
        annual_income = monthly_income * 12
        if annual_income > 60000 and remaining > 500:
            analysis += f"- Consider 401(k) contributions: ${min(remaining * 0.5, 2000):,.2f}/month\n"
            analysis += "- Tax-advantaged savings available\n"
            recommendations.append("Tax Advisor")
            
            # Add communication message
            comm_system.add_message(
                from_agent="Budget Planner",
                message=f"Client has ${remaining:,.2f} monthly surplus. Tax-advantaged savings recommended.",
                recommendations=["Tax Advisor"]
            )
        
        # Investment planning
        if remaining > 1000:
            analysis += f"- Investment capacity: ${remaining * 0.7:,.2f}/month\n"
            analysis += "- Consider systematic investment plans\n"
            recommendations.append("Investment Analyst")
            
            # Add communication message
            comm_system.add_message(
                from_agent="Budget Planner",
                message=f"Client has ${remaining:,.2f} available for investments. Portfolio planning needed.",
                recommendations=["Investment Analyst"]
            )
        
        # Goal-based recommendations
        if financial_goals:
            analysis += f"\nGOAL ANALYSIS:\n"
            for goal in financial_goals:
                goal_amount = goal.get('amount', 0)
                goal_months = goal.get('months', 12)
                monthly_needed = goal_amount / goal_months
                
                analysis += f"- {goal.get('name', 'Goal')}: ${monthly_needed:,.2f}/month needed\n"
                
                if monthly_needed > remaining:
                    analysis += f"  ⚠️ Exceeds available funds by ${monthly_needed - remaining:,.2f}\n"
        
        if recommendations:
            analysis += f"\n🤝 RECOMMENDED CONSULTATIONS:\n"
            for rec in set(recommendations):
                analysis += f"- {rec}: For specialized optimization\n"
        
        return analysis
    
    except Exception as e:
        return f"Error in budget analysis: {str(e)}"

@tool
def comprehensive_financial_planning(input_data: str) -> str:
    """
    Comprehensive financial planning that coordinates all aspects.
    
    Args:
        input_data: JSON string with complete financial picture
    
    Returns:
        Comprehensive financial plan with coordination recommendations
    """
    try:
        data = json.loads(input_data)
        monthly_income = data.get('monthly_income', 0)
        annual_income = monthly_income * 12
        age = data.get('age', 35)
        financial_goals = data.get('financial_goals', [])
        
        plan = f"""
COMPREHENSIVE FINANCIAL PLANNING:
- Age: {age} years
- Annual Income: ${annual_income:,.2f}
- Monthly Income: ${monthly_income:,.2f}

INTEGRATED RECOMMENDATIONS:
"""
        
        # Life stage recommendations
        if age < 30:
            plan += "- YOUNG PROFESSIONAL PHASE:\n"
            plan += "  * Focus on career growth and skill development\n"
            plan += "  * Build emergency fund (3-6 months expenses)\n"
            plan += "  * Start aggressive investment strategy\n"
        elif age < 50:
            plan += "- WEALTH BUILDING PHASE:\n"
            plan += "  * Maximize retirement contributions\n"
            plan += "  * Consider real estate investment\n"
            plan += "  * Tax optimization strategies\n"
        else:
            plan += "- PRE-RETIREMENT PHASE:\n"
            plan += "  * Focus on capital preservation\n"
            plan += "  * Estate planning considerations\n"
            plan += "  * Reduce portfolio risk\n"
        
        # Coordination requirements
        coordination_needed = []
        
        if annual_income > 75000:
            coordination_needed.append("Tax Advisor")
            plan += "\n- Tax optimization critical at this income level\n"
        
        if monthly_income > 5000:
            coordination_needed.append("Investment Analyst")
            plan += "- Investment strategy essential for wealth building\n"
        
        if financial_goals:
            plan += f"\n- {len(financial_goals)} financial goals require coordination\n"
        
        # Request multi-agent coordination
        if len(coordination_needed) > 1:
            comm_system.request_handoff(
                from_agent="Budget Planner",
                to_agent="Workflow Coordinator",
                reason="Multi-agent coordination needed for comprehensive planning",
                data={
                    'required_agents': coordination_needed,
                    'client_profile': data,
                    'priority': 'high' if annual_income > 100000 else 'medium'
                }
            )
            
            plan += f"\n🔄 WORKFLOW COORDINATION REQUESTED\n"
            plan += f"Agents needed: {', '.join(coordination_needed)}\n"
        
        return plan
    
    except Exception as e:
        return f"Error in comprehensive planning: {str(e)}"

# Test the enhanced budget tools
print("Testing Enhanced Budget Analysis:")
test_input = json.dumps({
    "monthly_income": 7500,
    "expenses": {
        "housing": 2000,
        "food": 800,
        "transportation": 600,
        "entertainment": 400,
        "utilities": 300
    },
    "financial_goals": [
        {"name": "Emergency Fund", "amount": 30000, "months": 12},
        {"name": "House Down Payment", "amount": 60000, "months": 36}
    ]
})
result = enhanced_budget_analysis.run(test_input)
print(result)
print("\n✓ Enhanced Budget tools created!")
print(f"✓ Total messages: {len(comm_system.messages)}")
print(f"✓ Total handoff requests: {len(comm_system.handoff_requests)}")

Testing Enhanced Budget Analysis:

ENHANCED BUDGET ANALYSIS:
- Monthly Income: $7,500.00
- Total Expenses: $4,100.00
- Available for Savings: $3,400.00
- Current Savings Rate: 45.3%

BUDGET OPTIMIZATION:
- EXCELLENT: Strong savings rate for financial growth
- Consider 401(k) contributions: $1,700.00/month
- Tax-advantaged savings available
- Investment capacity: $2,380.00/month
- Consider systematic investment plans

GOAL ANALYSIS:
- Emergency Fund: $2,500.00/month needed
- House Down Payment: $1,666.67/month needed

🤝 RECOMMENDED CONSULTATIONS:
- Investment Analyst: For specialized optimization
- Tax Advisor: For specialized optimization


✓ Enhanced Budget tools created!
✓ Total messages: 6
✓ Total handoff requests: 0


## Part 4: Creating Communicating Agents

Now let's create agents that can communicate with each other, make handoff requests, and coordinate their expertise through the communication system.

In [6]:
# Create Communicating Agents with Handoff Capabilities

def create_communicating_tax_advisor():
    """Create a Tax Advisor that can communicate and request handoffs."""
    
    tax_llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.1,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Enhanced tools with communication
    tax_tools = [enhanced_tax_analysis, tax_investment_coordination]
    
    tax_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a collaborative Tax Advisor who works as part of a financial planning team.
        
        Your expertise: Tax law, deductions, tax-efficient strategies
        
        Collaboration principles:
        - Identify when clients need Investment or Budget planning expertise
        - Recommend handoffs to other specialists when appropriate
        - Consider the full financial picture, not just taxes
        - Communicate findings that affect other areas
        
        When you identify investment or budgeting implications:
        - Explicitly recommend consulting the relevant specialist
        - Explain why the handoff would benefit the client
        - Provide context for the other agent
        
        Your tone: Professional, precise, collaborative"""),
        
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    tax_agent = create_tool_calling_agent(tax_llm, tax_tools, tax_prompt)
    return AgentExecutor(agent=tax_agent, tools=tax_tools, verbose=True, max_iterations=3)

def create_communicating_investment_analyst():
    """Create an Investment Analyst that can communicate and coordinate."""
    
    investment_llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.3,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Enhanced tools with communication
    investment_tools = [enhanced_portfolio_analysis, investment_budget_coordination]
    
    investment_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a collaborative Investment Analyst working in a financial planning team.
        
        Your expertise: Portfolio management, risk assessment, investment strategies
        
        Collaboration principles:
        - Consider tax implications of investment decisions
        - Assess budget constraints and savings capacity
        - Recommend handoffs when tax or budget optimization is needed
        - Coordinate with other specialists for comprehensive solutions
        
        When you identify tax or budget considerations:
        - Recommend consulting the Tax Advisor for tax-efficient strategies
        - Suggest Budget Planner consultation for affordability analysis
        - Explain the interconnections between investment and other financial areas
        
        Your tone: Analytical, forward-thinking, team-oriented"""),
        
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    investment_agent = create_tool_calling_agent(investment_llm, investment_tools, investment_prompt)
    return AgentExecutor(agent=investment_agent, tools=investment_tools, verbose=True, max_iterations=3)

def create_communicating_budget_planner():
    """Create a Budget Planner that can coordinate comprehensive planning."""
    
    budget_llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.4,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    
    # Enhanced tools with communication
    budget_tools = [enhanced_budget_analysis, comprehensive_financial_planning]
    
    budget_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a collaborative Budget Planner and Financial Coordinator on a planning team.
        
        Your expertise: Budgeting, financial goals, comprehensive financial planning
        
        Collaboration principles:
        - Coordinate between tax and investment strategies
        - Identify when multiple specialists need to work together
        - Create comprehensive financial plans that integrate all areas
        - Facilitate handoffs and ensure nothing falls through cracks
        
        Your special role as coordinator:
        - When complex situations arise, recommend multi-agent collaboration
        - Synthesize recommendations from different specialists
        - Ensure all aspects of financial planning are addressed
        
        Your tone: Encouraging, comprehensive, coordination-focused"""),
        
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    budget_agent = create_tool_calling_agent(budget_llm, budget_tools, budget_prompt)
    return AgentExecutor(agent=budget_agent, tools=budget_tools, verbose=True, max_iterations=3)

# Create the communicating agents
print("Creating Communicating Financial Agents...")
comm_tax_advisor = create_communicating_tax_advisor()
comm_investment_analyst = create_communicating_investment_analyst()
comm_budget_planner = create_communicating_budget_planner()

print("✅ Communicating agents created!")
print("  🤖 Tax Advisor - Can recommend Investment & Budget specialists")
print("  📈 Investment Analyst - Can coordinate with Tax & Budget teams")
print("  💰 Budget Planner - Can orchestrate comprehensive planning")
print("✅ Communication system active!")

Creating Communicating Financial Agents...


✅ Communicating agents created!
  🤖 Tax Advisor - Can recommend Investment & Budget specialists
  📈 Investment Analyst - Can coordinate with Tax & Budget teams
  💰 Budget Planner - Can orchestrate comprehensive planning
✅ Communication system active!


## Part 5: Communication & Workflow System

Now let's create a workflow system that can handle agent communications, process handoffs, and orchestrate multi-agent collaborations.

In [7]:
# Communication & Workflow Management System

class FinancialPlanningWorkflow:
    """Advanced workflow system with agent communication and handoffs."""
    
    def __init__(self):
        """Initialize the workflow system with communicating agents."""
        self.agents = {
            'Tax Advisor': comm_tax_advisor,
            'Investment Analyst': comm_investment_analyst,
            'Budget Planner': comm_budget_planner
        }
        self.communication = comm_system
        self.active_workflows = []
        
    def start_workflow(self, client_query: str, client_profile: dict = None):
        """Start a new workflow with potential multi-agent coordination."""
        
        workflow_id = len(self.active_workflows) + 1
        workflow = {
            'id': workflow_id,
            'query': client_query,
            'client_profile': client_profile or {},
            'steps': [],
            'recommendations': [],
            'active_agents': [],
            'status': 'started'
        }
        
        print(f"🚀 STARTING WORKFLOW #{workflow_id}")
        print(f"Query: {client_query}")
        
        # Determine initial agent based on query
        initial_agent = self._route_initial_query(client_query)
        workflow['active_agents'].append(initial_agent)
        
        print(f"📋 Initial Agent: {initial_agent}")
        
        # Get initial response
        response = self.agents[initial_agent].invoke({"input": client_query})
        
        workflow['steps'].append({
            'agent': initial_agent,
            'action': 'initial_analysis',
            'response': response['output']
        })
        
        # Check for handoff requests
        handoffs = self._check_for_handoffs(workflow_id)
        
        if handoffs:
            print(f"\n🔄 HANDOFFS DETECTED: {len(handoffs)} requests")
            workflow = self._process_handoffs(workflow, handoffs)
        
        self.active_workflows.append(workflow)
        return workflow
    
    def _route_initial_query(self, query: str) -> str:
        """Route the initial query to the most appropriate agent."""
        query_lower = query.lower()
        
        if any(keyword in query_lower for keyword in ['tax', 'deduction', 'irs', 'filing']):
            return 'Tax Advisor'
        elif any(keyword in query_lower for keyword in ['invest', 'portfolio', 'stocks', 'bonds', 'risk']):
            return 'Investment Analyst'
        else:
            return 'Budget Planner'  # Default coordinator
    
    def _check_for_handoffs(self, workflow_id: int) -> List[dict]:
        """Check for recent handoff requests from agents."""
        # Get recent handoff requests (in real system, would filter by workflow_id)
        recent_handoffs = [req for req in self.communication.handoff_requests[-5:]]
        return recent_handoffs
    
    def _process_handoffs(self, workflow: dict, handoffs: List[dict]) -> dict:
        """Process handoff requests and coordinate multiple agents."""
        
        for handoff in handoffs:
            to_agent = handoff['to_agent']
            
            if to_agent == 'Workflow Coordinator':
                # Handle multi-agent coordination
                workflow = self._coordinate_multiple_agents(workflow, handoff['data'])
            elif to_agent in self.agents and to_agent not in workflow['active_agents']:
                # Handle single agent handoff
                workflow = self._execute_handoff(workflow, handoff)
        
        return workflow
    
    def _execute_handoff(self, workflow: dict, handoff: dict) -> dict:
        """Execute a handoff to another agent."""
        
        to_agent = handoff['to_agent']
        reason = handoff['reason']
        data = handoff['data']
        
        print(f"\n📤 EXECUTING HANDOFF to {to_agent}")
        print(f"Reason: {reason}")
        
        # Create handoff query
        handoff_query = f"Handoff from {handoff['from_agent']}: {reason}. "
        handoff_query += f"Client data: {json.dumps(data, indent=2)}"
        
        # Get response from target agent
        response = self.agents[to_agent].invoke({"input": handoff_query})
        
        workflow['active_agents'].append(to_agent)
        workflow['steps'].append({
            'agent': to_agent,
            'action': 'handoff_response',
            'handoff_reason': reason,
            'response': response['output']
        })
        
        return workflow
    
    def _coordinate_multiple_agents(self, workflow: dict, coordination_data: dict) -> dict:
        """Coordinate multiple agents for comprehensive planning."""
        
        required_agents = coordination_data.get('required_agents', [])
        client_profile = coordination_data.get('client_profile', {})
        
        print(f"\n🎯 MULTI-AGENT COORDINATION")
        print(f"Required agents: {', '.join(required_agents)}")
        
        coordination_results = {}
        
        for agent_name in required_agents:
            if agent_name in self.agents and agent_name not in workflow['active_agents']:
                print(f"\n📊 Consulting {agent_name}...")
                
                # Create coordination query
                coord_query = f"Coordination request for comprehensive financial planning. "
                coord_query += f"Client profile: {json.dumps(client_profile, indent=2)}"
                
                response = self.agents[agent_name].invoke({"input": coord_query})
                coordination_results[agent_name] = response['output']
                
                workflow['active_agents'].append(agent_name)
                workflow['steps'].append({
                    'agent': agent_name,
                    'action': 'coordination_response',
                    'response': response['output']
                })
        
        return workflow
    
    def get_workflow_summary(self, workflow_id: int) -> str:
        """Get a summary of the workflow with all agent interactions."""
        
        if workflow_id > len(self.active_workflows):
            return "Workflow not found"
        
        workflow = self.active_workflows[workflow_id - 1]
        
        summary = f"""
🔍 WORKFLOW #{workflow['id']} SUMMARY
═══════════════════════════════════════

Original Query: {workflow['query']}
Active Agents: {', '.join(workflow['active_agents'])}
Total Steps: {len(workflow['steps'])}

AGENT INTERACTIONS:
"""
        
        for i, step in enumerate(workflow['steps'], 1):
            summary += f"\n{i}. {step['agent']} - {step['action'].replace('_', ' ').title()}\n"
            summary += f"   Response: {step['response'][:150]}...\n"
        
        # Communication summary
        summary += f"\n📞 COMMUNICATION SUMMARY:\n"
        summary += f"Messages exchanged: {len(self.communication.messages)}\n"
        summary += f"Handoffs executed: {len(self.communication.handoff_requests)}\n"
        
        return summary

# Create the workflow system
print("Creating Financial Planning Workflow System...")
workflow_system = FinancialPlanningWorkflow()
print("✅ Workflow system initialized!")
print("  🔄 Agent communication enabled")
print("  📤 Handoff processing ready")
print("  🎯 Multi-agent coordination available")
print("✅ Ready for complex financial planning workflows!")

Creating Financial Planning Workflow System...
✅ Workflow system initialized!
  🔄 Agent communication enabled
  📤 Handoff processing ready
  🎯 Multi-agent coordination available
✅ Ready for complex financial planning workflows!


## Part 6: Communication & Workflow Demonstrations

Let's test our advanced communication system with realistic scenarios that demonstrate agent handoffs, multi-agent coordination, and workflow management.

In [8]:
# Demonstration 1: Simple Agent Handoff
print("=" * 60)
print("DEMO 1: AGENT HANDOFF IN ACTION")
print("=" * 60)

# Start with a tax question that should trigger investment recommendations
tax_query = "I make $120,000 per year and have some stock investments. How can I minimize my tax burden?"

print(f"📋 Starting with Tax Advisor for: {tax_query}")
print()

# Clear previous communications for demo
comm_system.messages = []
comm_system.handoff_requests = []

# Start workflow
workflow_1 = workflow_system.start_workflow(tax_query)

print("\n" + "=" * 50)
print("WORKFLOW RESULTS:")
print("=" * 50)
print(workflow_system.get_workflow_summary(workflow_1['id']))

print(f"\n📊 Communication Messages Generated: {len(comm_system.messages)}")
for i, msg in enumerate(comm_system.messages, 1):
    print(f"{i}. From {msg['from_agent']}: {msg['message'][:80]}...")

print(f"\n🔄 Handoff Requests: {len(comm_system.handoff_requests)}")
for i, req in enumerate(comm_system.handoff_requests, 1):
    print(f"{i}. {req['from_agent']} → {req['to_agent']}: {req['reason']}")

print("\n✅ Demo 1 Complete: Agent handoff successfully demonstrated!")

DEMO 1: AGENT HANDOFF IN ACTION
📋 Starting with Tax Advisor for: I make $120,000 per year and have some stock investments. How can I minimize my tax burden?

🚀 STARTING WORKFLOW #1
Query: I make $120,000 per year and have some stock investments. How can I minimize my tax burden?
📋 Initial Agent: Tax Advisor


> Entering new AgentExecutor chain...



Invoking: `enhanced_tax_analysis` with `{'input_data': '{"income":120000,"filing_status":"single","deductions":10000}'}`



TAX ANALYSIS RESULTS:
- Gross Income: $120,000.00
- Filing Status: single
- Estimated Tax: $23,188.00
- Effective Rate: 19.3%

TAX OPTIMIZATION INSIGHTS:
- Potential monthly savings: $2,000.00
- Consider retirement account contributions
- High earner: Consider advanced strategies
- Estate planning may be beneficial

🤝 RECOMMENDED CONSULTATIONS:
- Investment Analyst: For specialized advice
- Budget Planner: For specialized advice
Based on your income of $120,000 per year and filing status as single, here are some insights to minimize your tax burden:

- Estimated Tax: $23,188.00
- Effective Tax Rate: 19.3%
- Potential monthly savings: $2,000.00
- Consider retirement account contributions
- High earner: Consider advanced tax strategies
- Estate planning may be beneficial

🤝 I recommend consulting with an Investment Analyst for specialized advice on optimizing your 

In [9]:
# Demonstration 2: Multi-Agent Coordination
print("\n" + "=" * 60)
print("DEMO 2: MULTI-AGENT COORDINATION")
print("=" * 60)

# Complex query that requires multiple specialists
complex_query = "I'm 35, earn $150,000, want to buy a house in 3 years, and need comprehensive financial planning."

print(f"📋 Complex Query: {complex_query}")
print()

# Clear communications for new demo
comm_system.messages = []
comm_system.handoff_requests = []

# Start workflow with complex requirements
workflow_2 = workflow_system.start_workflow(
    complex_query,
    client_profile={
        'age': 35,
        'annual_income': 150000,
        'monthly_income': 12500,
        'goals': ['house_purchase', 'retirement_planning', 'tax_optimization']
    }
)

print("\n" + "=" * 50)
print("MULTI-AGENT COORDINATION RESULTS:")
print("=" * 50)
print(workflow_system.get_workflow_summary(workflow_2['id']))

print(f"\n🎯 Agents Involved: {', '.join(workflow_2['active_agents'])}")
print(f"📊 Total Coordination Steps: {len(workflow_2['steps'])}")

print("\n🔄 Cross-Agent Communications:")
for i, msg in enumerate(comm_system.messages, 1):
    print(f"{i}. {msg['from_agent']} recommends: {', '.join(msg['recommendations'])}")
    print(f"   Message: {msg['message'][:100]}...")

print("\n✅ Demo 2 Complete: Multi-agent coordination successfully demonstrated!")


DEMO 2: MULTI-AGENT COORDINATION
📋 Complex Query: I'm 35, earn $150,000, want to buy a house in 3 years, and need comprehensive financial planning.

🚀 STARTING WORKFLOW #2
Query: I'm 35, earn $150,000, want to buy a house in 3 years, and need comprehensive financial planning.
📋 Initial Agent: Budget Planner


> Entering new AgentExecutor chain...



Invoking: `comprehensive_financial_planning` with `{'input_data': '{"age":35,"income":150000,"financial_goals":[{"goal":"buy a house","timeframe":"3 years"}]}'}`



COMPREHENSIVE FINANCIAL PLANNING:
- Age: 35 years
- Annual Income: $0.00
- Monthly Income: $0.00

INTEGRATED RECOMMENDATIONS:
- WEALTH BUILDING PHASE:
  * Maximize retirement contributions
  * Consider real estate investment
  * Tax optimization strategies

- 1 financial goals require coordination
I have started the comprehensive financial planning process for you. Here are some initial recommendations based on your information:

- Wealth Building Phase:
  - Maximize retirement contributions
  - Consider real estate investment
  - Implement tax optimization strategies

There is one financial goal identified that requires coordination. Let's continue with the planning to ensure all aspects are addressed.

> Finished chain.

MULTI-AGENT COORDINATION RESULTS:

🔍 WORKFLOW #2 SUMMARY
═══════════════════════════════════════

Ori

In [10]:
# Demonstration 3: Communication Pattern Analysis
print("\n" + "=" * 60)
print("DEMO 3: COMMUNICATION PATTERN ANALYSIS")
print("=" * 60)

def analyze_communication_patterns():
    """Analyze the communication patterns between agents."""
    
    print("🔍 ANALYZING AGENT COMMUNICATION PATTERNS:")
    print()
    
    # Analyze message patterns
    agent_communications = {}
    for msg in comm_system.messages:
        agent = msg['from_agent']
        if agent not in agent_communications:
            agent_communications[agent] = {'sent': 0, 'recommendations': []}
        
        agent_communications[agent]['sent'] += 1
        agent_communications[agent]['recommendations'].extend(msg['recommendations'])
    
    print("📊 MESSAGE STATISTICS:")
    for agent, stats in agent_communications.items():
        unique_recommendations = list(set(stats['recommendations']))
        print(f"  {agent}:")
        print(f"    - Messages sent: {stats['sent']}")
        print(f"    - Recommends: {', '.join(unique_recommendations) if unique_recommendations else 'None'}")
    
    # Analyze handoff patterns
    print(f"\n🔄 HANDOFF ANALYSIS:")
    handoff_matrix = {}
    for req in comm_system.handoff_requests:
        from_agent = req['from_agent']
        to_agent = req['to_agent']
        
        if from_agent not in handoff_matrix:
            handoff_matrix[from_agent] = {}
        if to_agent not in handoff_matrix[from_agent]:
            handoff_matrix[from_agent][to_agent] = 0
        
        handoff_matrix[from_agent][to_agent] += 1
    
    for from_agent, handoffs in handoff_matrix.items():
        print(f"  {from_agent} → ", end="")
        handoff_list = [f"{to_agent}({count})" for to_agent, count in handoffs.items()]
        print(", ".join(handoff_list))
    
    # Communication effectiveness
    print(f"\n✅ COMMUNICATION EFFECTIVENESS:")
    total_messages = len(comm_system.messages)
    total_handoffs = len(comm_system.handoff_requests)
    coordination_ratio = total_handoffs / total_messages if total_messages > 0 else 0
    
    print(f"  - Total inter-agent messages: {total_messages}")
    print(f"  - Successful handoffs: {total_handoffs}")
    print(f"  - Coordination efficiency: {coordination_ratio:.2f}")
    
    return {
        'total_messages': total_messages,
        'total_handoffs': total_handoffs,
        'coordination_ratio': coordination_ratio,
        'agent_communications': agent_communications,
        'handoff_matrix': handoff_matrix
    }

# Run communication analysis
analysis_results = analyze_communication_patterns()

print("\n✅ Demo 3 Complete: Communication patterns successfully analyzed!")


DEMO 3: COMMUNICATION PATTERN ANALYSIS
🔍 ANALYZING AGENT COMMUNICATION PATTERNS:

📊 MESSAGE STATISTICS:

🔄 HANDOFF ANALYSIS:

✅ COMMUNICATION EFFECTIVENESS:
  - Total inter-agent messages: 0
  - Successful handoffs: 0
  - Coordination efficiency: 0.00

✅ Demo 3 Complete: Communication patterns successfully analyzed!


## Summary: Agent Communication & Handoffs

Let's summarize the advanced concepts we've learned about agent communication, handoffs, and workflow orchestration.

In [11]:
# 🎯 AGENT COMMUNICATION CONCEPTS MASTERED

print("🔗 ADVANCED MULTI-AGENT COMMUNICATION CONCEPTS:")
print("=" * 60)

communication_concepts = {
    "Inter-Agent Communication": {
        "description": "Agents can send messages and share information",
        "example": "Tax Advisor informs Investment Analyst about tax implications",
        "implementation": "AgentCommunication.add_message()"
    },
    "Agent Handoffs": {
        "description": "Agents can request handoffs to other specialists",
        "example": "Investment Analyst requests Budget Planner for affordability analysis",
        "implementation": "AgentCommunication.request_handoff()"
    },
    "Workflow Orchestration": {
        "description": "System manages complex multi-agent workflows",
        "example": "Comprehensive financial planning requiring all three agents",
        "implementation": "FinancialPlanningWorkflow.start_workflow()"
    },
    "Smart Routing": {
        "description": "Initial queries routed to most appropriate agent",
        "example": "Tax questions → Tax Advisor, Investment questions → Investment Analyst",
        "implementation": "_route_initial_query()"
    },
    "Cross-Agent Recommendations": {
        "description": "Agents recommend consulting other specialists",
        "example": "High-income client triggers recommendations for tax optimization",
        "implementation": "Enhanced tool functions with recommendation logic"
    },
    "Communication Tracking": {
        "description": "Full audit trail of agent interactions",
        "example": "Track all messages, handoffs, and coordination requests",
        "implementation": "Communication system with timestamps and data"
    }
}

for concept, details in communication_concepts.items():
    print(f"\n🔹 {concept}:")
    print(f"   Description: {details['description']}")
    print(f"   Example: {details['example']}")
    print(f"   Implementation: {details['implementation']}")

print("\n" + "=" * 60)
print("✅ Advanced agent communication patterns mastered!")

# Key differences from Exercise 7
print("\n🔄 EVOLUTION FROM EXERCISE 7:")
print("Exercise 7: Independent specialized agents")
print("Exercise 8: Collaborative communicating agents")
print()
print("New capabilities added:")
print("- ✅ Agent-to-agent communication")
print("- ✅ Dynamic handoff requests")
print("- ✅ Workflow orchestration")
print("- ✅ Cross-agent recommendations")
print("- ✅ Communication pattern analysis")

🔗 ADVANCED MULTI-AGENT COMMUNICATION CONCEPTS:

🔹 Inter-Agent Communication:
   Description: Agents can send messages and share information
   Example: Tax Advisor informs Investment Analyst about tax implications
   Implementation: AgentCommunication.add_message()

🔹 Agent Handoffs:
   Description: Agents can request handoffs to other specialists
   Example: Investment Analyst requests Budget Planner for affordability analysis
   Implementation: AgentCommunication.request_handoff()

🔹 Workflow Orchestration:
   Description: System manages complex multi-agent workflows
   Example: Comprehensive financial planning requiring all three agents
   Implementation: FinancialPlanningWorkflow.start_workflow()

🔹 Smart Routing:
   Description: Initial queries routed to most appropriate agent
   Example: Tax questions → Tax Advisor, Investment questions → Investment Analyst
   Implementation: _route_initial_query()

🔹 Cross-Agent Recommendations:
   Description: Agents recommend consulting other 

In [12]:
# 🛠️ PRACTICAL IMPLEMENTATION GUIDE

print("=" * 60)
print("BUILDING YOUR OWN COMMUNICATING AGENT SYSTEM")
print("=" * 60)

implementation_steps = {
    1: {
        "title": "Design Communication System",
        "details": [
            "Create AgentCommunication class for message tracking",
            "Define message and handoff request structures",
            "Implement timestamp and data tracking"
        ]
    },
    2: {
        "title": "Enhance Agent Tools", 
        "details": [
            "Add communication capabilities to existing tools",
            "Include cross-agent recommendation logic",
            "Generate handoff requests when appropriate"
        ]
    },
    3: {
        "title": "Create Communicating Agents",
        "details": [
            "Modify agent prompts for collaboration",
            "Include handoff and coordination instructions",
            "Assign enhanced tools with communication"
        ]
    },
    4: {
        "title": "Build Workflow System",
        "details": [
            "Create workflow orchestration class",
            "Implement handoff processing logic",
            "Add multi-agent coordination capabilities"
        ]
    },
    5: {
        "title": "Test and Monitor",
        "details": [
            "Test various communication scenarios",
            "Monitor handoff success rates",
            "Analyze communication patterns for optimization"
        ]
    }
}

print("🔧 IMPLEMENTATION ROADMAP:")
for step, info in implementation_steps.items():
    print(f"\n{step}. {info['title']}:")
    for detail in info['details']:
        print(f"   • {detail}")

print("\n" + "=" * 60)
print("🎮 INTERACTIVE CHALLENGE")
print("=" * 60)

# Interactive example scenarios
challenge_scenarios = [
    "I'm 28, make $95K, have $50K in student loans, and want to start investing while optimizing taxes.",
    "I'm planning to retire in 15 years, have a 401k, and need help with estate planning and tax strategies.",
    "I want to start a business, need capital, and require comprehensive financial planning including tax implications.",
    "I'm getting married, combining finances, buying a house, and planning for children's education costs."
]

print("🎯 TRY THESE COMPLEX SCENARIOS:")
print("Each should trigger multiple agent coordination!\n")

for i, scenario in enumerate(challenge_scenarios, 1):
    print(f"{i}. {scenario}")

print(f"\n💡 CHALLENGE: Test these scenarios to see:")
print("  • Which agents get involved")
print("  • How handoffs are triggered")
print("  • Communication patterns that emerge")
print("  • Workflow orchestration in action")

print("\n" + "=" * 60)
print("🎉 CONGRATULATIONS!")
print("✅ Exercise 8 Complete: Agent Communication & Handoffs Mastered!")
print("🚀 You can now build sophisticated multi-agent systems with:")
print("   • Advanced inter-agent communication")
print("   • Dynamic handoff mechanisms") 
print("   • Workflow orchestration")
print("   • Cross-agent coordination")
print("=" * 60)

BUILDING YOUR OWN COMMUNICATING AGENT SYSTEM
🔧 IMPLEMENTATION ROADMAP:

1. Design Communication System:
   • Create AgentCommunication class for message tracking
   • Define message and handoff request structures
   • Implement timestamp and data tracking

2. Enhance Agent Tools:
   • Add communication capabilities to existing tools
   • Include cross-agent recommendation logic
   • Generate handoff requests when appropriate

3. Create Communicating Agents:
   • Modify agent prompts for collaboration
   • Include handoff and coordination instructions
   • Assign enhanced tools with communication

4. Build Workflow System:
   • Create workflow orchestration class
   • Implement handoff processing logic
   • Add multi-agent coordination capabilities

5. Test and Monitor:
   • Test various communication scenarios
   • Monitor handoff success rates
   • Analyze communication patterns for optimization

🎮 INTERACTIVE CHALLENGE
🎯 TRY THESE COMPLEX SCENARIOS:
Each should trigger multiple agent